In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join('..')))
from src import *
from itertools import product

import numpy as np
from matplotlib import pyplot as plt

In [8]:
moments = np.array([2, 3, 4])
ps = np.array([0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.15])
system_sizes = np.array([100, 200, 400, 800, 1600, 3200])#, 6400]
realizations = 10

# rework this to perform disorder average of log instead of log of disorder average

def produce_data(ps, system_sizes, moments, realizations, bins = 50):
    widths = [L // bins for L in system_sizes]
    fractal_dimension = np.zeros((len(ps), len(system_sizes), len(moments)))
    params = product(enumerate(ps), enumerate(system_sizes), enumerate(moments))
    for (i, p), (j, L), (k, m) in params:
        dimension_data_reduced = 0
        for _ in range(realizations):
            ness_correlation = C_NESS(0, 0, A(L, p))
            _, eigenvectors = np.linalg.eig(ness_correlation)
            moment_data = calculate_moments(eigenvectors.T, widths[j], m)
            dimension_data = map(
                lambda moment: generalized_fractal_dimension(moment, widths[j], m, L),
                moment_data
            )
            dimension_data_reduced += np.mean(list(dimension_data))
        fractal_dimension[i,j,k] = dimension_data_reduced / realizations
    return fractal_dimension

data = produce_data(ps, system_sizes, moments, realizations)
print(data)

[[[0.8395927  0.79567652 0.75416245]
  [0.88274603 0.84870486 0.81762471]
  [0.91346692 0.88460816 0.85944691]
  [0.93461669 0.91010415 0.88855879]
  [0.94734762 0.92741054 0.91024218]
  [0.9573553  0.94058713 0.92688804]]

 [[0.82638134 0.77334167 0.74308121]
  [0.8674626  0.82889696 0.79636569]
  [0.89669102 0.86002297 0.83538466]
  [0.91633223 0.8854079  0.86181905]
  [0.92736759 0.90374237 0.88291327]
  [0.9378685  0.91596231 0.89798054]]

 [[0.80889149 0.75951327 0.72650145]
  [0.84286308 0.80723118 0.77364106]
  [0.87502933 0.83376535 0.80521986]
  [0.89149946 0.85524644 0.82835754]
  [0.9017824  0.87031812 0.84588522]
  [0.91027379 0.88059589 0.85754855]]

 [[0.78523506 0.74290575 0.69766754]
  [0.8267546  0.78165486 0.74586072]
  [0.84663383 0.80170198 0.77521286]
  [0.86046464 0.81997565 0.78948637]
  [0.86720768 0.82719413 0.79972081]
  [0.86991183 0.8322251  0.80407319]]

 [[0.76479855 0.71799487 0.68008572]
  [0.79964275 0.74867863 0.7171462 ]
  [0.816251   0.76972388 0.736

In [34]:
%matplotlib qt

plt.rcParams.update({'font.size': 16})
fig, ax = plt.subplots(nrows=3, figsize=(10, 10), )
for (k, q) in enumerate(moments):
    for (i, p) in enumerate(ps):
        ax[k].scatter(system_sizes, data[i, :, k], 
                    linewidth=2.0, label="p = {}".format(p))
        ax[k].plot(system_sizes, data[i, :, k], linewidth=0.5)
    ax[k].set_xlabel(r"$L$", labelpad=-15)
    ax[k].set_ylabel(r"$D_q$")
    ax[k].set_xscale('log')
    ax[k].set_title(r"q={}".format(q))
    ax[k].legend(bbox_to_anchor=(1.0, 1.0))
plt.subplots_adjust(left = 0.1, right = 0.8, top = 0.95, bottom = 0.05, hspace = 0.4)
plt.show()